# Coordinate descent

CuML's `lasso` and `elastic net` implementations are both able to use the coordinate descent solver. Lasso extends linear regression by providing L1 regularization and elastic net extends linear regression by providing a combination of L1 and L2 regularizers.

A tremendous speed up can be demonstrated for datasets with a large number of rows and fewer columns. Furthermore, the mean squared error (MSE) value for cuML's implementation is much smaller than the Scikit-learn implementation on very small datasets.

The model can take array-like objects, either in host as NumPy arrays or in device (as Numba or cuda_array_interface-compliant), as well  as cuDF DataFrames. 

For information about cuDF, refer to the [cuDF documentation](https://docs.rapids.ai/api/cudf/stable/).

For information about cuML's lasso implementation: https://rapidsai.github.io/projects/cuml/en/stable/api.html#cuml.Lasso.

For information about cuML's elastic net implementation: https://rapidsai.github.io/projects/cuml/en/stable/api.html#cuml.ElasticNet.

## Imports

In [ ]:
import cudf
import numpy as np
from cuml import make_regression, train_test_split
from cuml.linear_model import ElasticNet as cuElasticNet, Lasso as cuLasso
from cuml.metrics.regression import r2_score
from sklearn.linear_model import ElasticNet as skElasticNet, Lasso as skLasso

## Define Parameters

In [ ]:
n_samples = 2**17
n_features = 500

learning_rate = 0.001
algorithm = "cyclic"
random_state=23

## Generate Data

In [ ]:
%%time
X, y = make_regression(n_samples=n_samples, n_features=n_features, random_state=random_state)

X = cudf.DataFrame.from_gpu_matrix(X)
y = cudf.DataFrame.from_gpu_matrix(y)[0]

X_cudf, X_cudf_test, y_cudf, y_cudf_test = train_test_split(X, y, test_size = 0.2, random_state=random_state)

In [ ]:
# Copy dataset from GPU memory to host memory.
# This is done to later compare CPU and GPU results.
X_train = X_cudf.to_pandas()
X_test = X_cudf_test.to_pandas()
y_train = y_cudf.to_pandas()
y_test = y_cudf_test.to_pandas()

## Lasso

### Scikit-learn Model

#### Fit, predict and evaluate

In [ ]:
%%time
ols_sk = skLasso(alpha=np.array([learning_rate]),
                 fit_intercept = True,
                 normalize = False,
                 max_iter = 1000,
                 selection=algorithm,
                 tol=1e-10)

ols_sk.fit(X_train, y_train)

In [ ]:
%%time
predict_sk = ols_sk.predict(X_test)

In [ ]:
%%time
r2_score_sk = r2_score(y_cudf_test, predict_sk)

### cuML Model

#### Fit, predict and evaluate

In [ ]:
%%time
ols_cuml = cuLasso(alpha=np.array([learning_rate]),
                   fit_intercept = True,
                   normalize = False,
                   max_iter = 1000,
                   selection=algorithm,
                   tol=1e-10)

ols_cuml.fit(X_cudf, y_cudf)

In [ ]:
%%time
predict_cuml = ols_cuml.predict(X_cudf_test).to_array()

In [ ]:
%%time
r2_score_cuml = r2_score(y_cudf_test, predict_cuml)

### Compare Results

In [ ]:
print("R^2 score (SKL):  %s" % r2_score_sk)
print("R^2 score (cuML): %s" % r2_score_cuml)

## Elastic Net

The elastic net model implemented in cuml contains the same parameters as the lasso model.
In addition to the variable values that can be altered in lasso, elastic net has another variable who's value can be changed: `l1_ratio` decides the ratio of amount of L1 and L2 regularization that would be applied to the model. When `l1_ratio = 0`, the model will have only L2 reqularization shall be applied to the model. (default = 0.5)

### Scikit-learn Model

#### Fit, predict and evaluate

In [ ]:
%%time
elastic_sk = skElasticNet(alpha=np.array([learning_rate]),
                          fit_intercept = True,
                          normalize = False,
                          max_iter = 1000,
                          selection=algorithm,
                          tol=1e-10)

elastic_sk.fit(X_train, y_train)

In [ ]:
%%time
predict_elas_sk = elastic_sk.predict(X_test)

In [ ]:
%%time
r2_score_elas_sk = r2_score(y_cudf_test, predict_elas_sk)

### cuML Model

#### Fit, predict and evaluate

In [ ]:
%%time
elastic_cuml = cuElasticNet(alpha=np.array([learning_rate]), 
                            fit_intercept = True,
                            normalize = False,
                            max_iter = 1000,
                            selection=algorithm,
                            tol=1e-10)

elastic_cuml.fit(X_cudf, y_cudf)

In [ ]:
%%time
predict_elas_cuml = elastic_cuml.predict(X_cudf_test).to_array()

In [ ]:
%%time
r2_score_elas_cuml = r2_score(y_cudf_test, predict_elas_cuml)

### Compare Results

In [ ]:
print("R^2 score (SKL):  %s" % r2_score_elas_sk)
print("R^2 score (cuML): %s" % r2_score_elas_cuml)